# 4 - Model building

## I have toyed a couple configurations of the model and decided to go with this stacked neural network
- Last 7 days features go into a RNN
- Last 7 days features go into a CNN
- Last 2 hours features go into a RNN
- Distance/datetime features go in a DNN
- Last 7th day features go into a DNN
- Outputs of all serve as metadata and feed into a DNN to predict 5 outputs

## Hyperparameter
- I tried to make this a great balance of complexity and accuracy
- The learning rate is difficult as model easily converges into a 'bad' local minima
- While I can't guarantee that I have got the global minima, but the results look pretty good to me

In [1]:
import numpy as np
import pandas as pd

from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.optimizers import *
from keras.losses import *
from keras import backend as K

import tensorflow as tf
print(tf.__version__)

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt

Using TensorFlow backend.


1.14.1-dev20190607


In [2]:
def root_mean_squared_error(y_true, y_pred):
    '''RMSE in keras'''
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

def symmetric_mean_absolute_mean_error(y_true, y_pred):
    '''SMAPE - modified (got it from kaggle)'''
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

def calculate_rmse(x, y):
    '''RMSE in numpy'''
    return np.sum(np.absolute(x - y))/x.shape[0]

def calculate_smape(y_true, y_pred):
    '''This is just a function wrapper lol'''
    return symmetric_mean_absolute_mean_error(y_true, y_pred)

In [3]:
train_df = pd.read_pickle('../data/preprocessed_train_df_for_training.pkl')
eval_df = pd.read_pickle('../data/preprocessed_eval_df_for_training.pkl')

In [4]:
raw_feat = [
    'distance_from_qp09d8', 'distance_from_qp03xx', 'distance_from_qp03wf',
    'day_of_week_sin', 'day_of_week_cos',
    'hour_sin', 'hour_cos',
    'fifteen_minute_sin', 'fifteen_minute_cos',
]

last_two_hour_feat = ['demand_t{0}'.format(t) for t in range(8)]

last_six_day_feat = []
for d in range(1,7):
    for i in range(13):
        last_six_day_feat.append('demand_{0}d_t{1}'.format(d, i))

last_week_today_feat = ['demand_7d_t{0}'.format(i) for i in range(13)]

target = ['target_1', 'target_2', 'target_3', 'target_4', 'target_5']

In [5]:
X_train_raw = train_df[raw_feat]
X_eval_raw = eval_df[raw_feat]

X_train_last_two_hour = train_df[last_two_hour_feat]
X_eval_last_two_hour = eval_df[last_two_hour_feat]

X_train_last_week = train_df[last_week_today_feat]
X_eval_last_week = eval_df[last_week_today_feat]

X_train_last_six_day = train_df[last_six_day_feat]
X_eval_last_six_day = eval_df[last_six_day_feat]


Y_train = train_df[target]
Y_eval = eval_df[target]

In [10]:
tf.set_random_seed(2019) # make sure keras will get reproducible result 

In [11]:
raw_inputs = Input(shape=(len(raw_feat),))
last_two_hour_inputs = Input(shape=(len(last_two_hour_feat),))
last_week_today_inputs = Input(shape=(len(last_week_today_feat),))
last_six_day_inputs = Input(shape=(len(last_six_day_feat),))

daily_cnn = concatenate([last_week_today_inputs, last_six_day_inputs])
daily_cnn = Reshape((7, len(last_week_today_feat),))(daily_cnn)
daily_cnn = Conv1D(16, 5, activation=relu, padding='same')(daily_cnn)
daily_cnn = MaxPool1D(2, 2, padding='same')(daily_cnn)
daily_cnn = Conv1D(8, 3, activation=relu, padding='same')(daily_cnn)
daily_cnn = MaxPool1D(2, 2, padding='same')(daily_cnn)
daily_cnn = Flatten()(daily_cnn)
daily_cnn = Model(inputs=[last_week_today_inputs, last_six_day_inputs], outputs=daily_cnn)

daily_rnn = concatenate([last_week_today_inputs, last_six_day_inputs])
daily_rnn = Reshape((7, len(last_week_today_feat),))(daily_rnn)
daily_rnn = GRU(32, return_sequences=True)(daily_rnn)
daily_rnn = GRU(16)(daily_rnn)
daily_rnn = Model(inputs=[last_week_today_inputs, last_six_day_inputs], outputs=daily_rnn)

raw_dnn = Dense(64, activation=relu)(raw_inputs)
raw_dnn = Dense(32, activation=relu)(raw_dnn)
raw_dnn = Dense(16, activation=relu)(raw_dnn)
raw_dnn = Model(inputs=raw_inputs, outputs=raw_dnn)

attention_dnn = Dense(64, activation=relu)(last_week_today_inputs)
attention_dnn = Dense(32, activation=relu)(attention_dnn)
attention_dnn = Dense(16, activation=relu)(attention_dnn)
attention_dnn = Model(inputs=last_week_today_inputs, outputs=attention_dnn)

last_two_hour_rnn = Reshape((len(last_two_hour_feat), 1, ))(last_two_hour_inputs)
last_two_hour_rnn = GRU(32, return_sequences=True)(last_two_hour_rnn)
last_two_hour_rnn = GRU(16)(last_two_hour_rnn)
last_two_hour_rnn = Model(inputs=last_two_hour_inputs, outputs=last_two_hour_rnn)

# last_week_today_inputs act as attention
final_stacked = concatenate([raw_dnn.output, 
                             daily_rnn.output, 
                             daily_cnn.output, 
                             attention_dnn.output, 
                             last_two_hour_rnn.output])
final_stacked = Dense(64, activation=relu)(final_stacked)
final_stacked = Dense(32, activation=relu)(final_stacked)
final_stacked = Dense(16, activation=relu)(final_stacked)
final_stacked = Dense(5, activation=hard_sigmoid)(final_stacked)
final_stacked = Model(inputs=[raw_inputs, last_week_today_inputs, last_six_day_inputs, last_two_hour_inputs],
                      outputs=final_stacked)

final_stacked.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 13)           0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 78)           0                                            
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 91)           0           input_11[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 7, 13)        0           concatenate_7[0][0]              
__________

In [13]:
# While Adagrad has great lr update, but this gives the model a little kick
for lr, ep in [(0.01, 1), (0.005, 1), (0.003, 1),
               (0.005, 1), (0.0003, 1), (0.0001, 1)]:
    final_stacked.compile(optimizer=Adagrad(lr=lr), loss=mean_squared_error, metrics=[root_mean_squared_error],)
    final_stacked.fit(
        [X_train_raw, X_train_last_week, X_train_last_six_day, X_train_last_two_hour], Y_train,
        batch_size=256, epochs=ep,
        validation_data=([X_eval_raw, X_eval_last_week, X_eval_last_six_day, X_eval_last_two_hour], Y_eval),
    )

Train on 5482125 samples, validate on 1395450 samples
Epoch 1/1
5482125/5482125 [==============================] - 360s 66us/step - loss: 8.5403e-04 - root_mean_squared_error: 0.0282 - val_loss: 8.9757e-04 - val_root_mean_squared_error: 0.0222
Train on 5482125 samples, validate on 1395450 samples
Epoch 1/1
5482125/5482125 [==============================] - 352s 64us/step - loss: 7.5571e-04 - root_mean_squared_error: 0.0271 - val_loss: 8.7603e-04 - val_root_mean_squared_error: 0.0219
Train on 5482125 samples, validate on 1395450 samples
Epoch 1/1
5482125/5482125 [==============================] - 359s 66us/step - loss: 7.4161e-04 - root_mean_squared_error: 0.0268 - val_loss: 8.7276e-04 - val_root_mean_squared_error: 0.0219
Train on 5482125 samples, validate on 1395450 samples
Epoch 1/1
5482125/5482125 [==============================] - 353s 64us/step - loss: 7.3873e-04 - root_mean_squared_error: 0.0268 - val_loss: 8.6265e-04 - val_root_mean_squared_error: 0.0218
Train on 5482125 samples

In [14]:
final_stacked.save('../model/final_stacked_80.h5')

In [15]:
stacked_rmse = []
for gh in eval_df.geohash6.unique():
    gh_df = eval_df[eval_df.geohash6 == gh]
    preds = final_stacked.predict([
        gh_df[raw_feat],
        gh_df[last_week_today_feat],
        gh_df[last_six_day_feat],
        gh_df[last_two_hour_feat]
    ])
    actual = gh_df[target]
    stacked_rmse.append({
        'geohash6': gh,
        'rmse(t+1)': calculate_rmse(actual['target_1'], preds[:, 0]),
        'rmse(t+2)': calculate_rmse(actual['target_2'], preds[:, 1]),
        'rmse(t+3)': calculate_rmse(actual['target_3'], preds[:, 2]),
        'rmse(t+4)': calculate_rmse(actual['target_4'], preds[:, 3]),
        'rmse(t+5)': calculate_rmse(actual['target_5'], preds[:, 4]),
        'smape(t+1)': calculate_smape(actual['target_1'], preds[:, 0]),
        'smape(t+2)': calculate_smape(actual['target_2'], preds[:, 1]),
        'smape(t+3)': calculate_smape(actual['target_3'], preds[:, 2]),
        'smape(t+4)': calculate_smape(actual['target_4'], preds[:, 3]),
        'smape(t+5)': calculate_smape(actual['target_5'], preds[:, 4]),
    })
stacked_df = pd.DataFrame(stacked_rmse)
stacked_df.to_csv('../metrics/stacked_model.csv', index=False)
stacked_df.describe()

,rmse(t+1),rmse(t+2),rmse(t+3),rmse(t+4),rmse(t+5),smape(t+1),smape(t+2),smape(t+3),smape(t+4),smape(t+5)
count,1329.000000,1.329000e+03,1329.000000,1.329000e+03,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000,1329.000000
mean,0.012436,1.376487e-02,0.014461,1.498439e-02,0.015406,51.652021,50.591721,50.100544,51.259864,50.680409
std,0.011131,1.274595e-02,0.013718,1.444920e-02,0.015114,26.546016,26.096379,26.335638,25.571516,25.539954
min,0.000000,1.047339e-07,0.000000,5.841255e-08,0.000000,0.000000,0.190476,0.000000,0.380952,0.000000
25%,0.003463,3.632281e-03,0.003660,3.785942e-03,0.003778,32.204681,31.699774,31.008934,32.578618,32.313670
50%,0.010054,1.071960e-02,0.011083,1.131061e-02,0.011445,51.602022,50.091757,50.462803,51.784532,51.727902
75%,0.018477,2.030096e-02,0.021047,2.158792e-02,0.022023,71.597790,69.367944,69.121369,70.181673,69.760072
max,0.062792,8.009573e-02,0.092854,1.016356e-01,0.111073,120.230463,121.409065,114.904273,110.065438,109.557381


## Generate predicted plots (Optional)
- This part of code will generate predicted values and actual values - just to verify pattern 
- Don't run if you are only looking to check the metrics

In [ ]:
!mkdir ../metrics/target_1
!mkdir ../metrics/target_2
!mkdir ../metrics/target_3
!mkdir ../metrics/target_4
!mkdir ../metrics/target_5

In [ ]:
for t in range(5):
    t2 = t + 1
    print(t, 4)
    for TEST_GEOHASH in eval_df.geohash6.unique():
        gh_df = eval_df[eval_df.geohash6 == TEST_GEOHASH]
        Y_preds = final_stacked.predict([
            gh_df[raw_feat],
            gh_df[last_week_today_feat],
            gh_df[last_six_day_feat],
            gh_df[last_two_hour_feat]
        ])
        figure(num=None, figsize=(25, 6))
        x_axis = np.arange(eval_df[eval_df.geohash6 == TEST_GEOHASH].shape[0])
        plt.plot(x_axis, Y_eval[eval_df.geohash6 == TEST_GEOHASH]['target_{0}'.format(t2)])
        plt.plot(x_axis, Y_preds[:, t])
        plt.savefig('../metrics/target_{0}/{1}.png'.format(t2, TEST_GEOHASH))
        plt.close('all')

## A more 'powerful' model (training 100% of the data)
- this is to evaluate the test/holdout dataset

In [16]:
df = pd.read_pickle('../data/preprocessed_df_for_training.pkl')
X_train_raw = df[raw_feat]
X_train_last_two_hour = df[last_two_hour_feat]
X_train_last_week = df[last_week_today_feat]
X_train_last_six_day = df[last_six_day_feat]
Y_train = df[target]

In [17]:
# del final_stacked
# final_stacked = load_model('../model/final_stacked_80.h5')
final_stacked.compile(optimizer=Adagrad(lr=3e-4), loss=mean_squared_error, metrics=[root_mean_squared_error],)
final_stacked.fit(
    [X_train_raw, X_train_last_week, X_train_last_six_day, X_train_last_two_hour], Y_train,
    batch_size=256, epochs=8)

Epoch 1/8
6877575/6877575 [==============================] - 443s 64us/step - loss: 7.5148e-04 - root_mean_squared_error: 0.0271
Epoch 2/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.5044e-04 - root_mean_squared_error: 0.0270
Epoch 3/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.5000e-04 - root_mean_squared_error: 0.0270
Epoch 4/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.4970e-04 - root_mean_squared_error: 0.0270
Epoch 5/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.4946e-04 - root_mean_squared_error: 0.0270
Epoch 6/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.4923e-04 - root_mean_squared_error: 0.0270
Epoch 7/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.4905e-04 - root_mean_squared_error: 0.0270
Epoch 8/8
6877575/6877575 [==============================] - 408s 59us/step - loss: 7.4888e-04 - 

In [18]:
final_stacked.save('../model/final_stacked_100.h5')